In [1]:
import os
import sys

src_path = os.path.abspath('../..')
print(src_path)
sys.path.append(src_path)

/Users/zzachw/Projects/14_nih/missing_data


In [2]:
from src.utils import create_directory, dump_pickle, set_seed, write_json

/Users/zzachw/Projects/14_nih/missing_data/src/utils.py:20: UserWarning: Unknown server name, using local path
  warnings.warn("Unknown server name, using local path")


In [3]:
local_data_path = os.path.join(src_path, "data")
local_data_path

'/Users/zzachw/Projects/14_nih/missing_data/data'

In [4]:
import csv
import json
from datetime import datetime, timedelta
from collections import Counter, OrderedDict
import random
import pandas as pd
import numpy as np
from itertools import chain
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
set_seed(seed=42)

In [6]:
output_path = os.path.join(local_data_path, "tadpole_challenge/processed")
create_directory(output_path)

In [7]:
TADPOLE_D1_D2 = pd.read_csv(os.path.join(local_data_path, "tadpole_challenge/TADPOLE_D1_D2.csv"), low_memory=False)
TADPOLE_D1_D2

,RID,PTID,VISCODE,SITE,D1,D2,COLPROT,ORIGPROT,EXAMDATE,DX_bl,...,PHASE_UPENNBIOMK9_04_19_17,BATCH_UPENNBIOMK9_04_19_17,KIT_UPENNBIOMK9_04_19_17,STDS_UPENNBIOMK9_04_19_17,RUNDATE_UPENNBIOMK9_04_19_17,ABETA_UPENNBIOMK9_04_19_17,TAU_UPENNBIOMK9_04_19_17,PTAU_UPENNBIOMK9_04_19_17,COMMENT_UPENNBIOMK9_04_19_17,update_stamp_UPENNBIOMK9_04_19_17
0,2,011_S_0002,bl,11,1,1,ADNI1,ADNI1,2005-09-08,CN,...,,,,,,,,,,
1,3,011_S_0003,bl,11,1,0,ADNI1,ADNI1,2005-09-12,AD,...,ADNI1,UPENNBIOMK9,P06-MP02-MP01,P06-MP02-MP01/2,2016-12-14,741.5,239.7,22.83,NaN,2017-04-20 14:39:54.0
2,3,011_S_0003,m06,11,1,0,ADNI1,ADNI1,2006-03-13,AD,...,,,,,,,,,,
3,3,011_S_0003,m12,11,1,0,ADNI1,ADNI1,2006-09-12,AD,...,ADNI1,UPENNBIOMK9,P06-MP02-MP01,P06-MP02-MP01/2,2016-12-14,601.4,251.7,24.18,NaN,2017-04-20 14:39:54.0
4,3,011_S_0003,m24,11,1,0,ADNI1,ADNI1,2007-09-12,AD,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12736,4167,116_S_4167,m60,116,1,1,ADNI2,ADNI2,2016-11-16,LMCI,...,,,,,,,,,,
12737,4199,116_S_4199,m60,116,1,1,ADNI2,ADNI2,2016-12-12,EMCI,...,,,,,,,,,,
12738,4557,053_S_4557,m60,53,1,1,ADNI2,ADNI2,2017-03-15,EMCI,...,,,,,,,,,,
12739,4512,100_S_4512,m48,100,1,1,ADNI2,ADNI2,2017-04-04,EMCI,...,,,,,,,,,,


In [8]:
TADPOLE_D1_D2.RID.nunique()

1737

In [9]:
TADPOLE_D1_D2["ID"] = TADPOLE_D1_D2.apply(lambda x: str(x.RID) + "-" + x.VISCODE, axis=1)
TADPOLE_D1_D2.ID.nunique()

12741

In [10]:
TADPOLE_D1_D2.head()

,RID,PTID,VISCODE,SITE,D1,D2,COLPROT,ORIGPROT,EXAMDATE,DX_bl,...,BATCH_UPENNBIOMK9_04_19_17,KIT_UPENNBIOMK9_04_19_17,STDS_UPENNBIOMK9_04_19_17,RUNDATE_UPENNBIOMK9_04_19_17,ABETA_UPENNBIOMK9_04_19_17,TAU_UPENNBIOMK9_04_19_17,PTAU_UPENNBIOMK9_04_19_17,COMMENT_UPENNBIOMK9_04_19_17,update_stamp_UPENNBIOMK9_04_19_17,ID
0,2,011_S_0002,bl,11,1,1,ADNI1,ADNI1,2005-09-08,CN,...,,,,,,,,,,2-bl
1,3,011_S_0003,bl,11,1,0,ADNI1,ADNI1,2005-09-12,AD,...,UPENNBIOMK9,P06-MP02-MP01,P06-MP02-MP01/2,2016-12-14,741.5,239.7,22.83,NaN,2017-04-20 14:39:54.0,3-bl
2,3,011_S_0003,m06,11,1,0,ADNI1,ADNI1,2006-03-13,AD,...,,,,,,,,,,3-m06
3,3,011_S_0003,m12,11,1,0,ADNI1,ADNI1,2006-09-12,AD,...,UPENNBIOMK9,P06-MP02-MP01,P06-MP02-MP01/2,2016-12-14,601.4,251.7,24.18,NaN,2017-04-20 14:39:54.0,3-m12
4,3,011_S_0003,m24,11,1,0,ADNI1,ADNI1,2007-09-12,AD,...,,,,,,,,,,3-m24


In [11]:
# 0 cn
# 1 mci
# 2 ad
DXCHANGE_map = {
    1: 0,
    2: 1,
    3: 2,
    4: 1,
    5: 2,
    6: 2,
    7: 0,
    8: 1,
    9: 0,
    float("nan"): float("nan")
}

In [12]:
TADPOLE_D1_D2.DXCHANGE.unique()

array([ 1.,  3.,  2.,  4.,  5.,  7.,  8., nan,  6.])

In [13]:
TADPOLE_D1_D2["LABEL"] = TADPOLE_D1_D2.DXCHANGE.map(DXCHANGE_map)
TADPOLE_D1_D2.head()

,RID,PTID,VISCODE,SITE,D1,D2,COLPROT,ORIGPROT,EXAMDATE,DX_bl,...,KIT_UPENNBIOMK9_04_19_17,STDS_UPENNBIOMK9_04_19_17,RUNDATE_UPENNBIOMK9_04_19_17,ABETA_UPENNBIOMK9_04_19_17,TAU_UPENNBIOMK9_04_19_17,PTAU_UPENNBIOMK9_04_19_17,COMMENT_UPENNBIOMK9_04_19_17,update_stamp_UPENNBIOMK9_04_19_17,ID,LABEL
0,2,011_S_0002,bl,11,1,1,ADNI1,ADNI1,2005-09-08,CN,...,,,,,,,,,2-bl,0.0
1,3,011_S_0003,bl,11,1,0,ADNI1,ADNI1,2005-09-12,AD,...,P06-MP02-MP01,P06-MP02-MP01/2,2016-12-14,741.5,239.7,22.83,NaN,2017-04-20 14:39:54.0,3-bl,2.0
2,3,011_S_0003,m06,11,1,0,ADNI1,ADNI1,2006-03-13,AD,...,,,,,,,,,3-m06,2.0
3,3,011_S_0003,m12,11,1,0,ADNI1,ADNI1,2006-09-12,AD,...,P06-MP02-MP01,P06-MP02-MP01/2,2016-12-14,601.4,251.7,24.18,NaN,2017-04-20 14:39:54.0,3-m12,2.0
4,3,011_S_0003,m24,11,1,0,ADNI1,ADNI1,2007-09-12,AD,...,,,,,,,,,3-m24,2.0


In [14]:
label_df = TADPOLE_D1_D2[["ID", "LABEL"]]
label_df = label_df.dropna()
label_df = label_df.set_index("ID")
y = label_df.to_dict()["LABEL"]
write_json(os.path.join(output_path, "y.json"), y)

In [15]:
TADPOLE_D1_D2 = TADPOLE_D1_D2.replace(" ", float("nan"))
TADPOLE_D1_D2

,RID,PTID,VISCODE,SITE,D1,D2,COLPROT,ORIGPROT,EXAMDATE,DX_bl,...,KIT_UPENNBIOMK9_04_19_17,STDS_UPENNBIOMK9_04_19_17,RUNDATE_UPENNBIOMK9_04_19_17,ABETA_UPENNBIOMK9_04_19_17,TAU_UPENNBIOMK9_04_19_17,PTAU_UPENNBIOMK9_04_19_17,COMMENT_UPENNBIOMK9_04_19_17,update_stamp_UPENNBIOMK9_04_19_17,ID,LABEL
0,2,011_S_0002,bl,11,1,1,ADNI1,ADNI1,2005-09-08,CN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2-bl,0.0
1,3,011_S_0003,bl,11,1,0,ADNI1,ADNI1,2005-09-12,AD,...,P06-MP02-MP01,P06-MP02-MP01/2,2016-12-14,741.5,239.7,22.83,NaN,2017-04-20 14:39:54.0,3-bl,2.0
2,3,011_S_0003,m06,11,1,0,ADNI1,ADNI1,2006-03-13,AD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3-m06,2.0
3,3,011_S_0003,m12,11,1,0,ADNI1,ADNI1,2006-09-12,AD,...,P06-MP02-MP01,P06-MP02-MP01/2,2016-12-14,601.4,251.7,24.18,NaN,2017-04-20 14:39:54.0,3-m12,2.0
4,3,011_S_0003,m24,11,1,0,ADNI1,ADNI1,2007-09-12,AD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3-m24,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12736,4167,116_S_4167,m60,116,1,1,ADNI2,ADNI2,2016-11-16,LMCI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4167-m60,NaN
12737,4199,116_S_4199,m60,116,1,1,ADNI2,ADNI2,2016-12-12,EMCI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4199-m60,NaN
12738,4557,053_S_4557,m60,53,1,1,ADNI2,ADNI2,2017-03-15,EMCI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4557-m60,NaN
12739,4512,100_S_4512,m48,100,1,1,ADNI2,ADNI2,2017-04-04,EMCI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4512-m48,NaN


In [16]:
TADPOLE_D1_D2_Dict = pd.read_csv(os.path.join(local_data_path, "tadpole_challenge/TADPOLE_D1_D2_Dict.csv"), low_memory=False)
TADPOLE_D1_D2_Dict

,ID,FLDNAME,TBLNAME,CRFNAME,QUESTION,TEXT,STATUS,DEPRULE,TYPE,LENGTH,...,DE,MC,VISIBLE,LOGVISIBLE,MTBLNAME,SUBTBLNAME,POPJSCPT,GRID,DEF,update_stamp
0,-4,RID,ADNIMERGE,Key ADNI tables merged into one table,-4,Participant roster ID,-4,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,2013-01-22 14:36:06.0
1,-4,PTID,ADNIMERGE,Key ADNI tables merged into one table,-4,Participant ID,-4,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,2013-01-22 14:36:06.0
2,-4,VISCODE,ADNIMERGE,Key ADNI tables merged into one table,-4,Visit code,-4,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,2013-01-22 14:36:06.0
3,,D1,,,,Denotes whether participant belongs to dataset D1,,,,,...,,,,,,,,,,
4,,D2,,,,Denotes whether participant belongs to dataset D2,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1913,NaN,RUNDATE_UPENNBIOMK9_04_19_17,UPENNBIOMK9,UPENN CSF Biomarkers Elecsys,-4,Date of analytical run,-4,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,2017-04-20 14:39:44.0
1914,NaN,ABETA_UPENNBIOMK9_04_19_17,UPENNBIOMK9,UPENN CSF Biomarkers Elecsys,-4,Result,-4,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,2017-04-20 14:39:44.0
1915,NaN,TAU_UPENNBIOMK9_04_19_17,UPENNBIOMK9,UPENN CSF Biomarkers Elecsys,-4,Result,-4,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,2017-04-20 14:39:44.0
1916,NaN,PTAU_UPENNBIOMK9_04_19_17,UPENNBIOMK9,UPENN CSF Biomarkers Elecsys,-4,Result,-4,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,2017-04-20 14:39:44.0


In [17]:
TADPOLE_D1_D2_Dict.UNITS = TADPOLE_D1_D2_Dict.UNITS.replace('-4', float('nan'))
TADPOLE_D1_D2_Dict.UNITS = TADPOLE_D1_D2_Dict.UNITS.replace(' ', float('nan'))
TADPOLE_D1_D2_Dict.UNITS = TADPOLE_D1_D2_Dict.UNITS.replace('UNITS', float('nan'))
TADPOLE_D1_D2_Dict.UNITS.unique()

array([nan, 'mm3', 'mm2', 'mm', 'Normalized Counts', 'Percentage',
       'unitless', 'av45 uptake - reference region',
       'av45 uptake in cortical grey matter',
       'av45 ratio (cortical grey matter/whole cerebellum)', 'binary',
       'av45 ratio (cortical grey matter/composite ref region)', 'mm^3',
       'av45 uptake - subregion', 'av1451 uptake - reference region',
       'av1451 uptake - subregion', 'FA', 'MD', 'RD', 'AD', 'pg/ml'],
      dtype=object)

In [18]:
TADPOLE_D1_D2_Dict.CODE = TADPOLE_D1_D2_Dict.CODE.replace('-4', float('nan'))
TADPOLE_D1_D2_Dict.CODE = TADPOLE_D1_D2_Dict.CODE.replace(' ', float('nan'))
TADPOLE_D1_D2_Dict.CODE = TADPOLE_D1_D2_Dict.CODE.replace('CODE', float('nan'))
TADPOLE_D1_D2_Dict.CODE.unique()

array([nan, '0 = No, 1 = Yes', 'Pass/Fail/Partial', 'Pass/Fail'],
      dtype=object)

In [19]:
# TADPOLE_D1_D2_Dict = TADPOLE_D1_D2_Dict.dropna(subset=["UNITS", "CODE"], how="all")
TADPOLE_D1_D2_Dict = TADPOLE_D1_D2_Dict.dropna(subset=["UNITS"], how="all")
TADPOLE_D1_D2_Dict

,ID,FLDNAME,TBLNAME,CRFNAME,QUESTION,TEXT,STATUS,DEPRULE,TYPE,LENGTH,...,DE,MC,VISIBLE,LOGVISIBLE,MTBLNAME,SUBTBLNAME,POPJSCPT,GRID,DEF,update_stamp
117,NaN,ST100SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,UCSFFSL,Longitudinal FreeSurfer (FreeSurfer Version 4.4),-4,Volume (WM Parcellation) of RightNonWMHypoInte...,-4,-4,N,8,...,-4,0,-4,-4,-4,-4,-4,-4,-4,2013-11-06 08:50:25.0
118,NaN,ST101SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,UCSFFSL,Longitudinal FreeSurfer (FreeSurfer Version 4.4),-4,Volume (WM Parcellation) of RightPallidum,-4,-4,N,8,...,-4,0,-4,-4,-4,-4,-4,-4,-4,2013-11-06 08:50:25.0
119,NaN,ST102CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,UCSFFSL,Longitudinal FreeSurfer (FreeSurfer Version 4.4),-4,Volume (Cortical Parcellation) of RightParacen...,-4,-4,N,5,...,-4,0,-4,-4,-4,-4,-4,-4,-4,2013-11-06 08:50:25.0
120,NaN,ST102SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,UCSFFSL,Longitudinal FreeSurfer (FreeSurfer Version 4.4),-4,Surface Area of RightParacentral,-4,-4,N,5,...,-4,0,-4,-4,-4,-4,-4,-4,-4,2013-11-06 08:50:25.0
121,NaN,ST102TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16,UCSFFSL,Longitudinal FreeSurfer (FreeSurfer Version 4.4),-4,Cortical Thickness Average of RightParacentral,-4,-4,N,4,...,-4,0,-4,-4,-4,-4,-4,-4,-4,2013-11-06 08:50:25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1902,-4,AD_SUMCC_DTIROI_04_30_14,DTIROI,DTI ROI summary measures,-4,Bilateral full corpus callosum,-4,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,2014-05-01 10:26:46.0
1903,-4,AD_SUMFX_DTIROI_04_30_14,DTIROI,DTI ROI summary measures,-4,Bilateral fornix,-4,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,2014-05-01 10:26:46.0
1914,NaN,ABETA_UPENNBIOMK9_04_19_17,UPENNBIOMK9,UPENN CSF Biomarkers Elecsys,-4,Result,-4,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,2017-04-20 14:39:44.0
1915,NaN,TAU_UPENNBIOMK9_04_19_17,UPENNBIOMK9,UPENN CSF Biomarkers Elecsys,-4,Result,-4,-4,-4,-4,...,-4,-4,-4,-4,-4,-4,-4,-4,-4,2017-04-20 14:39:44.0


In [20]:
fields = [
    "UCSFFSL_02_01_16",
    "UCSFFSL51ALL_08_01_16",
    "BAIPETNMRC_09_12_16",
    "UCBERKELEYAV45_10_17_16",
    "UCBERKELEYAV1451_10_17_16",
    "DTIROI_04_30_14",
    "UPENNBIOMK9_04_19_17",
]

In [21]:
field_to_column = {}
for field in fields:
    field_to_column[field] = [c for c in TADPOLE_D1_D2_Dict.FLDNAME if (field in c) and (c in TADPOLE_D1_D2)]
field_to_column

{'UCSFFSL_02_01_16': ['ST100SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST101SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST102CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST102SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST102TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST102TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST103CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST103SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST103TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST103TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST104CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST104SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST104TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST104TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST105CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST105SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST105TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST105TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16',
  'ST106CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_

In [22]:
for f in field_to_column:
    print(f)
    print(len(field_to_column[f]))

UCSFFSL_02_01_16
346
UCSFFSL51ALL_08_01_16
346
BAIPETNMRC_09_12_16
334
UCBERKELEYAV45_10_17_16
237
UCBERKELEYAV1451_10_17_16
242
DTIROI_04_30_14
228
UPENNBIOMK9_04_19_17
3


In [23]:
for f in field_to_column:
    df = TADPOLE_D1_D2[field_to_column[f]]
    condition = df.apply(lambda row: row.isna().any() == row.isna().all(), axis=1).all()
    print(condition)

False
False
False
False
False
True
False


In [24]:
df = TADPOLE_D1_D2[field_to_column["UCSFFSL_02_01_16"]].dropna(how="all")
df = df.replace("-4", float("nan"))
nan_ratios = df.isnull().sum() / len(df)

# Sort the columns based on their NaN ratios
sorted_nan_ratios = nan_ratios.sort_values(ascending=False)

# Print the sorted result
for col, ratio in sorted_nan_ratios.items():
    print(f"{col}: {ratio:.2%}")

ST100SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST126SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST81TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST81SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST81CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST28CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST87CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST33SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST22TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST22TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST22SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST22CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST92SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST122SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST81TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST67SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST63SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST41SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST8SV_U

In [25]:
df = TADPOLE_D1_D2[field_to_column["UCSFFSL51ALL_08_01_16"]].dropna(how="all")
df = df.replace("-4", float("nan"))
nan_ratios = df.isnull().sum() / len(df)

# Sort the columns based on their NaN ratios
sorted_nan_ratios = nan_ratios.sort_values(ascending=False)

# Print the sorted result
for col, ratio in sorted_nan_ratios.items():
    print(f"{col}: {ratio:.2%}")

ST100SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST126SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST81TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST81SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST81CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST28CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST87CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST33SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST22TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST22TA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST22SA_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST22CV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST92SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST122SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST81TS_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST67SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST63SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST41SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16: 100.00%
ST8SV_U

In [26]:
df = TADPOLE_D1_D2[field_to_column["BAIPETNMRC_09_12_16"]].dropna(how="all")
df = df.replace("-4", float("nan"))
nan_ratios = df.isnull().sum() / len(df)

# Sort the columns based on their NaN ratios
sorted_nan_ratios = nan_ratios.sort_values(ascending=False)

# Print the sorted result
for col, ratio in sorted_nan_ratios.items():
    print(f"{col}: {ratio:.2%}")

TMPSUPL05_BAIPETNMRC_09_12_16: 100.00%
PARIINFR04_BAIPETNMRC_09_12_16: 100.00%
PRECUNR04_BAIPETNMRC_09_12_16: 100.00%
CINGANT04_BAIPETNMRC_09_12_16: 100.00%
CINGMID04_BAIPETNMRC_09_12_16: 100.00%
OCCMIDL04_BAIPETNMRC_09_12_16: 100.00%
OCCSUPL04_BAIPETNMRC_09_12_16: 100.00%
OCCSUPR04_BAIPETNMRC_09_12_16: 100.00%
OCCINFL04_BAIPETNMRC_09_12_16: 100.00%
OCCINFR04_BAIPETNMRC_09_12_16: 100.00%
THALAML04_BAIPETNMRC_09_12_16: 100.00%
THALAMR04_BAIPETNMRC_09_12_16: 100.00%
FRTMEDL04_BAIPETNMRC_09_12_16: 100.00%
FRTMEDR04_BAIPETNMRC_09_12_16: 100.00%
SUPMRGR04_BAIPETNMRC_09_12_16: 100.00%
ANGULL04_BAIPETNMRC_09_12_16: 100.00%
FRTMIDL05_BAIPETNMRC_09_12_16: 100.00%
FRTMIDR05_BAIPETNMRC_09_12_16: 100.00%
FRTSUPL05_BAIPETNMRC_09_12_16: 100.00%
FRTSUPR05_BAIPETNMRC_09_12_16: 100.00%
FRTSMEDL05_BAIPETNMRC_09_12_16: 100.00%
FRTSMEDR05_BAIPETNMRC_09_12_16: 100.00%
TMPINFL05_BAIPETNMRC_09_12_16: 100.00%
FUSFRML05_BAIPETNMRC_09_12_16: 100.00%
FUSFRMR05_BAIPETNMRC_09_12_16: 100.00%
PRECUNL04_BAIPETNMRC_09

In [27]:
df = TADPOLE_D1_D2[field_to_column["UCBERKELEYAV45_10_17_16"]].dropna(how="all")
df = df.replace("-4", float("nan"))
nan_ratios = df.isnull().sum() / len(df)

# Sort the columns based on their NaN ratios
sorted_nan_ratios = nan_ratios.sort_values(ascending=False)

# Print the sorted result
for col, ratio in sorted_nan_ratios.items():
    print(f"{col}: {ratio:.2%}")

VENTRICLE_5TH_SIZE_UCBERKELEYAV45_10_17_16: 66.86%
VENTRICLE_5TH_UCBERKELEYAV45_10_17_16: 66.86%
RIGHT_VESSEL_SIZE_UCBERKELEYAV45_10_17_16: 0.19%
RIGHT_VESSEL_UCBERKELEYAV45_10_17_16: 0.19%
LEFT_VESSEL_UCBERKELEYAV45_10_17_16: 0.14%
LEFT_VESSEL_SIZE_UCBERKELEYAV45_10_17_16: 0.14%
CTX_RH_ROSTRALANTERIORCINGULATE_UCBERKELEYAV45_10_17_16: 0.05%
CTX_RH_ROSTRALANTERIORCINGULATE_SIZE_UCBERKELEYAV45_10_17_16: 0.05%
CTX_LH_PARAHIPPOCAMPAL_SIZE_UCBERKELEYAV45_10_17_16: 0.05%
CTX_LH_PARAHIPPOCAMPAL_UCBERKELEYAV45_10_17_16: 0.05%
CTX_RH_ROSTRALMIDDLEFRONTAL_UCBERKELEYAV45_10_17_16: 0.00%
CTX_RH_SUPERIORTEMPORAL_SIZE_UCBERKELEYAV45_10_17_16: 0.00%
CTX_RH_ROSTRALMIDDLEFRONTAL_SIZE_UCBERKELEYAV45_10_17_16: 0.00%
CTX_RH_SUPERIORFRONTAL_UCBERKELEYAV45_10_17_16: 0.00%
CTX_RH_SUPERIORFRONTAL_SIZE_UCBERKELEYAV45_10_17_16: 0.00%
CTX_RH_SUPERIORPARIETAL_UCBERKELEYAV45_10_17_16: 0.00%
CTX_RH_SUPERIORPARIETAL_SIZE_UCBERKELEYAV45_10_17_16: 0.00%
CTX_RH_SUPERIORTEMPORAL_UCBERKELEYAV45_10_17_16: 0.00%
CEREBELLU

In [28]:
df = TADPOLE_D1_D2[field_to_column["UCBERKELEYAV1451_10_17_16"]].dropna(how="all")
df = df.replace("-4", float("nan"))
nan_ratios = df.isnull().sum() / len(df)

# Sort the columns based on their NaN ratios
sorted_nan_ratios = nan_ratios.sort_values(ascending=False)

# Print the sorted result
for col, ratio in sorted_nan_ratios.items():
    print(f"{col}: {ratio:.2%}")

BRAINSTEM_UCBERKELEYAV1451_10_17_16: 62.92%
BRAINSTEM_SIZE_UCBERKELEYAV1451_10_17_16: 62.92%
CTX_RH_POSTERIORCINGULATE_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_PRECENTRAL_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_PRECENTRAL_SIZE_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_PRECUNEUS_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_PRECUNEUS_SIZE_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_ROSTRALANTERIORCINGULATE_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_ROSTRALANTERIORCINGULATE_SIZE_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_ROSTRALMIDDLEFRONTAL_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_ROSTRALMIDDLEFRONTAL_SIZE_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_SUPERIORFRONTAL_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_SUPERIORFRONTAL_SIZE_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_SUPERIORPARIETAL_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_SUPERIORPARIETAL_SIZE_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_SUPERIORTEMPORAL_UCBERKELEYAV1451_10_17_16: 0.00%
CEREBELLUMGREYMATTER_UCBERKELEYAV1451_10_17_16: 0.00%
CTX_RH_SUPERIORTEMPORAL_SIZ

In [29]:
df = TADPOLE_D1_D2[field_to_column["DTIROI_04_30_14"]].dropna(how="all")
df = df.replace("-4", float("nan"))
nan_ratios = df.isnull().sum() / len(df)

# Sort the columns based on their NaN ratios
sorted_nan_ratios = nan_ratios.sort_values(ascending=False)

# Print the sorted result
for col, ratio in sorted_nan_ratios.items():
    print(f"{col}: {ratio:.2%}")

FA_CST_L_DTIROI_04_30_14: 0.00%
RD_SLF_R_DTIROI_04_30_14: 0.00%
RD_SFO_R_DTIROI_04_30_14: 0.00%
RD_IFO_L_DTIROI_04_30_14: 0.00%
RD_IFO_R_DTIROI_04_30_14: 0.00%
RD_SS_L_DTIROI_04_30_14: 0.00%
RD_SS_R_DTIROI_04_30_14: 0.00%
RD_EC_L_DTIROI_04_30_14: 0.00%
RD_EC_R_DTIROI_04_30_14: 0.00%
RD_UNC_L_DTIROI_04_30_14: 0.00%
RD_UNC_R_DTIROI_04_30_14: 0.00%
RD_FX_L_DTIROI_04_30_14: 0.00%
RD_FX_R_DTIROI_04_30_14: 0.00%
RD_GCC_L_DTIROI_04_30_14: 0.00%
RD_GCC_R_DTIROI_04_30_14: 0.00%
RD_BCC_L_DTIROI_04_30_14: 0.00%
RD_BCC_R_DTIROI_04_30_14: 0.00%
RD_SCC_L_DTIROI_04_30_14: 0.00%
RD_SCC_R_DTIROI_04_30_14: 0.00%
RD_RLIC_L_DTIROI_04_30_14: 0.00%
RD_RLIC_R_DTIROI_04_30_14: 0.00%
RD_TAP_L_DTIROI_04_30_14: 0.00%
RD_TAP_R_DTIROI_04_30_14: 0.00%
RD_SUMGCC_DTIROI_04_30_14: 0.00%
RD_SUMBCC_DTIROI_04_30_14: 0.00%
RD_SUMSCC_DTIROI_04_30_14: 0.00%
RD_SUMCC_DTIROI_04_30_14: 0.00%
RD_SFO_L_DTIROI_04_30_14: 0.00%
RD_SLF_L_DTIROI_04_30_14: 0.00%
FA_CST_R_DTIROI_04_30_14: 0.00%
RD_FX_ST_R_DTIROI_04_30_14: 0.00%
RD_ICP_

In [30]:
TADPOLE_D1_D2.EXAMDATE = pd.to_datetime(TADPOLE_D1_D2.EXAMDATE)
tadpole_grouped = TADPOLE_D1_D2.groupby("RID").apply(lambda x:(x["EXAMDATE"]-x["EXAMDATE"].min()).dt.days/365.25 + x["AGE"].min())
tadpole_grouped.sort_index(inplace=True)
TADPOLE_D1_D2.sort_values(by=["RID", "EXAMDATE"], inplace=True)
TADPOLE_D1_D2["AGE_AT_EXAM"] = tadpole_grouped.values
TADPOLE_D1_D2['AGE_INT'] = TADPOLE_D1_D2['AGE_AT_EXAM'].apply(int)

In [31]:
base = [
    # cog_tests_attributes
    "CDRSB", "ADAS11", "MMSE", "RAVLT_immediate",
    # mri_measures
    'Hippocampus', 'WholeBrain', 'Entorhinal', 'MidTemp' , "FDG", "AV45",
    # pet_measures
    "FDG", "AV45",
    # csf_measures
    "ABETA_UPENNBIOMK9_04_19_17", "TAU_UPENNBIOMK9_04_19_17", "PTAU_UPENNBIOMK9_04_19_17",
    # risk_factors
    "APOE4", "AGE",
    # age
    'AGE_AT_EXAM',
]
df = TADPOLE_D1_D2[base].dropna(how="all")
df = df.replace("-4", float("nan"))
nan_ratios = df.isnull().sum() / len(df)

# Sort the columns based on their NaN ratios
sorted_nan_ratios = nan_ratios.sort_values(ascending=False)

# Print the sorted result
for col, ratio in sorted_nan_ratios.items():
    print(f"{col}: {ratio:.2%}")

AV45: 83.38%
AV45: 83.38%
PTAU_UPENNBIOMK9_04_19_17: 81.40%
ABETA_UPENNBIOMK9_04_19_17: 81.39%
TAU_UPENNBIOMK9_04_19_17: 81.39%
FDG: 73.69%
FDG: 73.69%
Entorhinal: 49.23%
MidTemp: 49.23%
Hippocampus: 46.61%
WholeBrain: 39.65%
RAVLT_immediate: 30.69%
ADAS11: 30.07%
MMSE: 29.90%
CDRSB: 29.65%
APOE4: 0.09%
AGE: 0.00%
AGE_AT_EXAM: 0.00%


In [32]:
for f in fields:
    df = TADPOLE_D1_D2[["ID"] + field_to_column[f]].dropna(subset=field_to_column[f], how="all")
    df = df.replace("-4", float("nan"))
    df = df.dropna(how="any", axis=1)
    print(f, df.shape)
#     df = df.set_index("ID")
#     df = df.astype(float)
#     normalized_df = (df - df.mean()) / df.std()
#     dict_output = {index: row.tolist() for index, row in normalized_df.iterrows()}

UCSFFSL_02_01_16 (4341, 311)
UCSFFSL51ALL_08_01_16 (4341, 311)
BAIPETNMRC_09_12_16 (3858, 1)
UCBERKELEYAV45_10_17_16 (2118, 228)
UCBERKELEYAV1451_10_17_16 (89, 241)
DTIROI_04_30_14 (788, 229)
UPENNBIOMK9_04_19_17 (2372, 1)


In [33]:
df = TADPOLE_D1_D2[field_to_column["UCSFFSL_02_01_16"] + field_to_column["UCSFFSL51ALL_08_01_16"]].dropna(how="all")
df = df.dropna(how="any", axis=1)
print(df.shape)

(4341, 620)


In [34]:
field_to_column["UCSFFSL"] = field_to_column["UCSFFSL_02_01_16"] + field_to_column["UCSFFSL51ALL_08_01_16"]

In [35]:
new_fields = [
    "UCSFFSL",
    "UCBERKELEYAV45_10_17_16",
    "DTIROI_04_30_14",
]

In [36]:
for f in new_fields:
    df = TADPOLE_D1_D2[["ID"] + field_to_column[f]].dropna(subset=field_to_column[f], how="all")
    df = df.dropna(how="any", axis=1)
    print(f, df.shape)
    df = df.set_index("ID")
    df = df.astype(float)
    normalized_df = (df - df.mean()) / df.std()
#     dict_output = {index: row.tolist() for index, row in normalized_df.iterrows()}

UCSFFSL (4341, 621)
UCBERKELEYAV45_10_17_16 (2118, 228)
DTIROI_04_30_14 (788, 229)


In [37]:
for f in new_fields:
    df = TADPOLE_D1_D2[["ID"] + field_to_column[f]].dropna(subset=field_to_column[f], how="all")
    df = df.dropna(how="any", axis=1)
    df = df.set_index("ID")
    df = df.astype(float)
    normalized_df = (df - df.mean()) / df.std()
    print(f, normalized_df.isna().sum().sum())

UCSFFSL 0
UCBERKELEYAV45_10_17_16 0
DTIROI_04_30_14 0


In [38]:
for f in new_fields:
    df = TADPOLE_D1_D2[["ID"] + field_to_column[f]].dropna(subset=field_to_column[f], how="all")
    df = df.replace("-4", float("nan"))
    df = df.dropna(how="any", axis=1)
    print(f, df.shape)
    df = df.set_index("ID")
    df = df.astype(float)
    normalized_df = (df - df.mean()) / df.std()
    print(f, normalized_df.isna().sum().sum())

UCSFFSL (4341, 621)
UCSFFSL 0
UCBERKELEYAV45_10_17_16 (2118, 228)
UCBERKELEYAV45_10_17_16 0
DTIROI_04_30_14 (788, 229)
DTIROI_04_30_14 0


In [39]:
for f in new_fields:
    df = TADPOLE_D1_D2[["ID"] + field_to_column[f]].dropna(subset=field_to_column[f], how="all")
    df = df.dropna(how="any", axis=1)
    print(f, df.shape)
    df = df.set_index("ID")
    df = df.astype(float)
    normalized_df = (df - df.mean()) / df.std()
    dict_output = {index: row.tolist() for index, row in normalized_df.iterrows()}
    fn = os.path.join(output_path, f"x_{f}.json")
    print(fn)
    write_json(fn, dict_output)

UCSFFSL (4341, 621)
/Users/zzachw/Projects/14_nih/missing_data/data/tadpole_challenge/processed/x_UCSFFSL.json
UCBERKELEYAV45_10_17_16 (2118, 228)
/Users/zzachw/Projects/14_nih/missing_data/data/tadpole_challenge/processed/x_UCBERKELEYAV45_10_17_16.json
DTIROI_04_30_14 (788, 229)
/Users/zzachw/Projects/14_nih/missing_data/data/tadpole_challenge/processed/x_DTIROI_04_30_14.json


In [40]:
TADPOLE_D1_D2.PTGENDER.unique()

array(['Male', 'Female'], dtype=object)

In [41]:
TADPOLE_D1_D2.PTETHCAT.unique()

array(['Not Hisp/Latino', 'Hisp/Latino', 'Unknown'], dtype=object)

In [42]:
TADPOLE_D1_D2.PTRACCAT.unique()

array(['White', 'More than one', 'Black', 'Asian', 'Am Indian/Alaskan',
       'Unknown', 'Hawaiian/Other PI'], dtype=object)

In [43]:
TADPOLE_D1_D2.EXAMDATE = pd.to_datetime(TADPOLE_D1_D2.EXAMDATE)
tadpole_grouped = TADPOLE_D1_D2.groupby("RID").apply(lambda x:(x["EXAMDATE"]-x["EXAMDATE"].min()).dt.days/365.25 + x["AGE"].min())
tadpole_grouped.sort_index(inplace=True)
TADPOLE_D1_D2.sort_values(by=["RID", "EXAMDATE"], inplace=True)
TADPOLE_D1_D2["AGE_AT_EXAM"] = tadpole_grouped.values
TADPOLE_D1_D2['AGE_INT'] = TADPOLE_D1_D2['AGE_AT_EXAM'].apply(int)

In [44]:
TADPOLE_D1_D2

,RID,PTID,VISCODE,SITE,D1,D2,COLPROT,ORIGPROT,EXAMDATE,DX_bl,...,RUNDATE_UPENNBIOMK9_04_19_17,ABETA_UPENNBIOMK9_04_19_17,TAU_UPENNBIOMK9_04_19_17,PTAU_UPENNBIOMK9_04_19_17,COMMENT_UPENNBIOMK9_04_19_17,update_stamp_UPENNBIOMK9_04_19_17,ID,LABEL,AGE_AT_EXAM,AGE_INT
0,2,011_S_0002,bl,11,1,1,ADNI1,ADNI1,2005-09-08,CN,...,NaN,NaN,NaN,NaN,NaN,NaN,2-bl,0.0,74.300000,74
5723,2,011_S_0002,m06,11,1,1,ADNI1,ADNI1,2006-03-06,CN,...,NaN,NaN,NaN,NaN,NaN,NaN,2-m06,0.0,74.790075,74
5724,2,011_S_0002,m36,11,1,1,ADNI1,ADNI1,2008-08-27,CN,...,NaN,NaN,NaN,NaN,NaN,NaN,2-m36,0.0,77.267830,77
5725,2,011_S_0002,m60,11,1,1,ADNIGO,ADNI1,2010-09-22,CN,...,NaN,NaN,NaN,NaN,NaN,NaN,2-m60,0.0,79.337645,79
5726,2,011_S_0002,m66,11,1,1,ADNIGO,ADNI1,2011-03-04,CN,...,NaN,NaN,NaN,NaN,NaN,NaN,2-m66,NaN,79.783915,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10802,5295,057_S_5295,bl,57,1,1,ADNI2,ADNI2,2013-12-11,SMC,...,2017-01-13,416.8,88.69,8.53,NaN,2017-04-20 14:39:56.0,5295-bl,0.0,75.500000,75
10803,5295,057_S_5295,m03,57,1,1,ADNI2,ADNI2,2014-02-14,SMC,...,NaN,NaN,NaN,NaN,NaN,NaN,5295-m03,NaN,75.677960,75
12277,5295,057_S_5295,m24,57,1,1,ADNI2,ADNI2,2015-12-08,SMC,...,NaN,NaN,NaN,NaN,NaN,NaN,5295-m24,0.0,77.490418,77
10804,5296,053_S_5296,bl,53,1,1,ADNI2,ADNI2,2013-12-18,SMC,...,2016-11-28,1460,224.9,20.9,NaN,2017-04-20 14:39:56.0,5296-bl,0.0,69.300000,69


In [45]:
demo = TADPOLE_D1_D2[["ID", "AGE_INT", "PTGENDER", "PTRACCAT"]]
demo

,ID,AGE_INT,PTGENDER,PTRACCAT
0,2-bl,74,Male,White
5723,2-m06,74,Male,White
5724,2-m36,77,Male,White
5725,2-m60,79,Male,White
5726,2-m66,79,Male,White
...,...,...,...,...
10802,5295-bl,75,Female,White
10803,5295-m03,75,Female,White
12277,5295-m24,77,Female,White
10804,5296-bl,69,Male,White


In [46]:
demo = demo.dropna()
demo

,ID,AGE_INT,PTGENDER,PTRACCAT
0,2-bl,74,Male,White
5723,2-m06,74,Male,White
5724,2-m36,77,Male,White
5725,2-m60,79,Male,White
5726,2-m66,79,Male,White
...,...,...,...,...
10802,5295-bl,75,Female,White
10803,5295-m03,75,Female,White
12277,5295-m24,77,Female,White
10804,5296-bl,69,Male,White


In [47]:
demo = demo.set_index("ID")
demo = demo.to_dict("index")
demo

{'2-bl': {'AGE_INT': 74, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '2-m06': {'AGE_INT': 74, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '2-m36': {'AGE_INT': 77, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '2-m60': {'AGE_INT': 79, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '2-m66': {'AGE_INT': 79, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '2-m72': {'AGE_INT': 80, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '2-m78': {'AGE_INT': 80, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '2-m84': {'AGE_INT': 81, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '2-m90': {'AGE_INT': 81, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '2-m96': {'AGE_INT': 82, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '2-m102': {'AGE_INT': 82, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '2-m108': {'AGE_INT': 83, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '2-m120': {'AGE_INT': 84, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '3-bl': {'AGE_INT': 81, 'PTGENDER': 'Male', 'PTRACCAT': 'White'},
 '3-m06': {'AGE_INT': 81, 'PTGENDER': 'Male', '

In [48]:
write_json(os.path.join(output_path, "demo.json"), demo)